# Subjectivity classification with CNNs

In this notebook we implement the approched described in this [paper](https://arxiv.org/pdf/1408.5882.pdf) for classifiying sentences using Convolutional Neural Networks. In particular, we will classify sentences into "subjective" or "objective". 

## Subjectivity Dataset

The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [1]:
from pathlib import Path
PATH = Path("/Users/yinterian/teaching/deeplearning/data/rotten_imdb/")
list(PATH.iterdir())

[PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.300d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.100d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.50d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/plot.tok.gt9.5000'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/subjdata.README.1.0'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/quote.tok.gt9.5000'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.200d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.zip'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/rotten_imdb.tar')]

From the readme file:
- quote.tok.gt9.5000 contains 5000 subjective sentences (or snippets)
- plot.tok.gt9.5000 contains 5000 objective sentences

In [2]:
! head /Users/yinterian/teaching/deeplearning/data/rotten_imdb/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 
spurning her mother's insistence that she get on with her life , mary is thrown out of the house , rejected by joe , and expelled from school as she grows larger with child . 
amitabh can't believe the board of directors and his mind is filled with revenge and what better revenge than robbing the bank himself , ironic as it may sound . 
she , among others excentricities , talks to a small rock , gertrude , like if she was alive . 
this gives the girls a fair chance of pulling the wool over their eyes using their sexiness to poach any last vestige of common sense the dons might have had . 
styled after vh1's " behin

## String cleaning functions

In [3]:
import numpy as np
from collections import defaultdict
import re

In [4]:
def read_file(path):
    """ Read file returns a shuttled list.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = np.array(f.readlines())
    return content

In [5]:
def get_vocab(list_of_content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for content in list_of_content:
        for line in content:
            line = line.strip()
            words = set(line.split())
            for word in words:
                vocab[word] += 1
    return vocab       

## Split train and test

In [6]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip() for line in sub_content])
obj_content = np.array([line.strip() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

In [10]:
# getting vocab from training sets
data_vocab = get_vocab([X_train])

In [11]:
#data_vocab

## Embedding Layer

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
# an Embedding module containing 10 (words) tensors of size 3
embed = nn.Embedding(10, 3)
a = torch.LongTensor([[1,2,4,5,1]])
embed(a)

tensor([[[ 0.6611, -0.0256,  0.5207],
         [ 0.3977, -0.6290, -2.2998],
         [-0.7043,  0.2253,  0.5196],
         [ 0.0529, -2.0081,  0.0011],
         [ 0.6611, -0.0256,  0.5207]]])

In [14]:
## here is the randomly initialized embeddings
embed.weight.data

tensor([[ 2.0298,  0.9666,  1.3254],
        [ 0.6611, -0.0256,  0.5207],
        [ 0.3977, -0.6290, -2.2998],
        [ 0.0154,  0.1524, -0.1961],
        [-0.7043,  0.2253,  0.5196],
        [ 0.0529, -2.0081,  0.0011],
        [ 0.2850, -0.0970, -1.7501],
        [ 0.7354,  0.2891,  0.9144],
        [ 0.3188,  0.3426, -0.6627],
        [ 1.4081, -0.9440, -1.0364]])

### Initializing embedding layer with Glove embeddings

To get glove pre-trained embeddings:
    `wget http://nlp.stanford.edu/data/glove.6B.zip`

In this section I am keeping the whole Glove embeddings. You can decide to keep just the words on your training set.

In [15]:
! head -2 /Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392


We would like to initialize the embeddings from our model with the pre-trained Glove embeddings. After initializing we should "freeze" the embeddings at least initially. The rationale is that we first want the network to learn weights for the other parameters that were randomly initialize. After that phase we could finetune the embeddings to our task. 

`embed.weight.requires_grad = False` freezes the embedding parameters.

The following code initializes the embedding. Here `V` is the vocabulary size and `D` is the embedding size. `pretrained_weight` is a numpy matrix of shape `(V, D)`.

In [16]:
def loadGloveModel(gloveFile=PATH/"glove.6B.300d.txt"):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [17]:
word_vecs = loadGloveModel()

In [18]:
print(len(word_vecs.keys()), len(data_vocab.keys()))

400000 21416


In [19]:
def delete_rare_words(word_vecs, data_vocab, min_df=2):
    """ Deletes rare words from data_vocab
    
    Deletes words from data_vocab if they are not in word_vecs
    and don't have at least min_df occurrencies in data_vocab.
    """
    words_delete = []
    for word in data_vocab:
        if data_vocab[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: data_vocab.pop(word)
    return data_vocab

In [20]:
len(data_vocab.keys())

21416

In [21]:
# clean up issues here
data_vocab = delete_rare_words(word_vecs, data_vocab, min_df=2) 

In [22]:
len(data_vocab.keys())

18756

In [23]:
def create_embedding_matrix(word_vecs, data_vocab, min_df=2, D=300):
    """Creates embedding matrix from word vectors. """
    data_vocab = delete_rare_words(word_vecs, data_vocab, min_df)
    V = len(data_vocab.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, D), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(D, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, D)
    vocab2index["UNK"] = 1
    i = 2
    for word in data_vocab:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25,D)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [24]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, data_vocab)

In [25]:
len(pretrained_weight) # note that index 0 is for padding

18758

In [26]:
D = 300
V = len(pretrained_weight)
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2008, -0.0298,  0.2068,  ..., -0.0982, -0.0032,  0.0163],
        [-0.3144, -0.3239,  0.0450,  ..., -0.3788,  0.3690,  0.1244],
        ...,
        [-0.1369, -0.0570, -0.1921,  ..., -0.2036, -0.4955, -0.2766],
        [-0.3170, -0.4958,  0.3020,  ..., -0.1990,  0.0607, -0.1257],
        [ 0.2028, -0.3397, -0.1055,  ...,  0.5841, -0.4893,  0.0245]])

Question: How many parameters do we have in this embedding matrix?

## Encoding training and validation sets

We will be using 1D Convolutional neural networks as our model. CNNs assume a fixed input size so we need to assume a fixed size and truncate or pad the sentences as needed. Let's find a good value to set our sequence length to.

In [27]:
x_len = np.array([len(x.split()) for x in X_train])

In [28]:
np.percentile(x_len, 95) # let set the max sequence len to N=40

43.0

In [29]:
X_train[0]

'will god let her fall or give her a new path ?'

In [30]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("will", vocab2index["UNK"])

4

In [31]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 4, 11,  3, 12,  2,  5,  9, 12,  8, 10,  6,  7])

In [32]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [33]:
encode_sentence(X_train[0])

array([ 4, 11,  3, 12,  2,  5,  9, 12,  8, 10,  6,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [34]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [36]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Playing and debugging CNN layers

In [37]:
V = len(pretrained_weight)
D = 300
N = 40

In [38]:
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2008, -0.0298,  0.2068,  ..., -0.0982, -0.0032,  0.0163],
        [-0.3144, -0.3239,  0.0450,  ..., -0.3788,  0.3690,  0.1244],
        ...,
        [-0.1369, -0.0570, -0.1921,  ..., -0.2036, -0.4955, -0.2766],
        [-0.3170, -0.4958,  0.3020,  ..., -0.1990,  0.0607, -0.1257],
        [ 0.2028, -0.3397, -0.1055,  ...,  0.5841, -0.4893,  0.0245]])

In [39]:
x = x_train[:2]
x.shape

(2, 40)

In [40]:
x = torch.LongTensor(x)
x

tensor([[  4,  11,   3,  12,   2,   5,   9,  12,   8,  10,   6,   7,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [ 25,  20,  27,  29,  13,  28,  19,  14,  30,  31,  36,  16,
          18,  17,  28,  24,  32,  21,  33,  23,  22,  26,  34,  15,
          35,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0]])

In [41]:
x1 = emb(x)
x1.shape

torch.Size([2, 40, 300])

In [42]:
x1 = x1.transpose(1,2)  # needs to convert x to (batch, embedding_dim, sentence_len)
x1.size()

torch.Size([2, 300, 40])

In [43]:
conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)

In [44]:
x3 = conv_3(x1)

In [45]:
x3.size()

torch.Size([2, 100, 38])

In [46]:
conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)

In [47]:
x4 = conv_4(x1)
x5 = conv_5(x1)
print(x4.size(), x5.size())

torch.Size([2, 100, 37]) torch.Size([2, 100, 36])


Note that the convolution all apply to the same `x1`. How do we combine now the results of the convolutions? 

In [48]:
# 100 3-gram detectors
x3 = nn.ReLU()(x3)
x3 = nn.MaxPool1d(kernel_size = 38)(x3)
x3.size()

torch.Size([2, 100, 1])

In [49]:
# 100 4-gram detectors
x4 = nn.ReLU()(x4)
x4 = nn.MaxPool1d(kernel_size = 37)(x4)
x4.size()

torch.Size([2, 100, 1])

In [50]:
# 100 5-gram detectors
x5 = nn.ReLU()(x5)
x5 = nn.MaxPool1d(kernel_size = 36)(x5)
x5.size()

torch.Size([2, 100, 1])

In [51]:
# concatenate x3, x4, x5
out = torch.cat([x3, x4, x5], 2)
out.size()

torch.Size([2, 100, 3])

In [52]:
out = out.view(out.size(0), -1)
out.size()

torch.Size([2, 300])

After this we have a fully connected network. Let's write a network that implements this.

## 1D CNN model for sentence classification

Notation:
* V -- vocabulary size
* D -- embedding size
* N -- MAX Sentence length

In [53]:
class SentenceCNN(nn.Module):
    
    def __init__(self, V, D, glove_weights):
        super(SentenceCNN, self).__init__()
        self.glove_weights = glove_weights
        self.embedding = nn.Embedding(V, D, padding_idx=0)
        self.embedding.weight.data.copy_(torch.from_numpy(self.glove_weights))
        self.embedding.weight.requires_grad = False ## freeze embeddings

        self.conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(300, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 38)(x3)
        x4 = nn.MaxPool1d(kernel_size = 37)(x4)
        x5 = nn.MaxPool1d(kernel_size = 36)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        return self.fc(out)   

In [54]:
V = len(pretrained_weight)
D = 300
N = 40
model = SentenceCNN(V, D, glove_weights=pretrained_weight)

In [55]:
# testing the model
x = x_train[:10]
print(x.shape)
x = torch.LongTensor(x)

(10, 40)


In [56]:
y_hat = model(x)
y_hat.size()

torch.Size([10, 1])

## Training

Note that I am not bodering with mini-batches since our dataset is small.

In [58]:
model = SentenceCNN(V, D, glove_weights=pretrained_weight) #.cuda()

In [59]:
def val_metrics(m):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    y_hat = m(x)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [60]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.6978036761283875, 0.5059999823570251)

In [61]:
# this filters parameters with p.requires_grad=True
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [62]:
def train_epocs(model, epochs=10, lr=0.01):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    model.train()
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, accuracy = val_metrics(model)
        print("train loss %.3f test loss %.3f and accuracy %.3f" % 
              (loss.item(), val_loss, accuracy))

In [70]:
model = SentenceCNN(V, D, glove_weights=pretrained_weight)

In [71]:
train_epocs(model, epochs=10, lr=0.005)

train loss 0.708 test loss 1.228 and accuracy 0.494
train loss 1.174 test loss 0.425 and accuracy 0.867
train loss 0.410 test loss 0.677 and accuracy 0.614
train loss 0.681 test loss 0.550 and accuracy 0.696
train loss 0.549 test loss 0.397 and accuracy 0.841
train loss 0.383 test loss 0.379 and accuracy 0.858
train loss 0.349 test loss 0.423 and accuracy 0.811
train loss 0.381 test loss 0.455 and accuracy 0.785
train loss 0.406 test loss 0.449 and accuracy 0.788
train loss 0.399 test loss 0.414 and accuracy 0.817


In [72]:
# how to figure out the parameters
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


### Unfreezing the embeddings

In [73]:
# unfreezing the embeddings
model.embedding.weight.requires_grad = True

In [74]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([18758, 300]), torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


In [75]:
train_epocs(model, epochs=10, lr=0.001)

train loss 0.364 test loss 0.359 and accuracy 0.855
train loss 0.315 test loss 0.340 and accuracy 0.870
train loss 0.300 test loss 0.334 and accuracy 0.880
train loss 0.295 test loss 0.326 and accuracy 0.882
train loss 0.286 test loss 0.314 and accuracy 0.882
train loss 0.269 test loss 0.301 and accuracy 0.879
train loss 0.251 test loss 0.292 and accuracy 0.877
train loss 0.235 test loss 0.290 and accuracy 0.878
train loss 0.223 test loss 0.291 and accuracy 0.877
train loss 0.213 test loss 0.289 and accuracy 0.876


In [76]:
train_epocs(model, epochs=10, lr=0.001)

train loss 0.206 test loss 0.270 and accuracy 0.890
train loss 0.200 test loss 0.265 and accuracy 0.891
train loss 0.191 test loss 0.260 and accuracy 0.886
train loss 0.176 test loss 0.263 and accuracy 0.887
train loss 0.168 test loss 0.264 and accuracy 0.885
train loss 0.162 test loss 0.257 and accuracy 0.889
train loss 0.152 test loss 0.248 and accuracy 0.894
train loss 0.142 test loss 0.241 and accuracy 0.900
train loss 0.135 test loss 0.237 and accuracy 0.900
train loss 0.127 test loss 0.234 and accuracy 0.903


## Whithout pretrain emmbeddings

In [77]:
class SentenceCNN2(nn.Module):
    
    def __init__(self, V, D):
        super(SentenceCNN2, self).__init__()
        self.embedding = nn.Embedding(V, D, padding_idx=0)

        self.conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)
        
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(300, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 38)(x3)
        x4 = nn.MaxPool1d(kernel_size = 37)(x4)
        x5 = nn.MaxPool1d(kernel_size = 36)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        return self.fc(out)   

In [78]:
V = len(pretrained_weight)
model = SentenceCNN2(V, D=100)

In [79]:
train_epocs(model, epochs=10, lr=0.005)

train loss 0.781 test loss 1.481 and accuracy 0.506
train loss 1.464 test loss 0.795 and accuracy 0.507
train loss 0.778 test loss 0.666 and accuracy 0.550
train loss 0.655 test loss 0.856 and accuracy 0.505
train loss 0.838 test loss 0.723 and accuracy 0.542
train loss 0.703 test loss 0.532 and accuracy 0.739
train loss 0.510 test loss 0.529 and accuracy 0.729
train loss 0.487 test loss 0.618 and accuracy 0.632
train loss 0.559 test loss 0.604 and accuracy 0.650
train loss 0.538 test loss 0.513 and accuracy 0.734


In [80]:
train_epocs(model, epochs=10, lr=0.001)

train loss 0.450 test loss 0.452 and accuracy 0.794
train loss 0.399 test loss 0.453 and accuracy 0.807
train loss 0.403 test loss 0.456 and accuracy 0.793
train loss 0.404 test loss 0.445 and accuracy 0.808
train loss 0.387 test loss 0.432 and accuracy 0.815
train loss 0.369 test loss 0.425 and accuracy 0.817
train loss 0.358 test loss 0.425 and accuracy 0.803
train loss 0.351 test loss 0.427 and accuracy 0.796
train loss 0.350 test loss 0.424 and accuracy 0.797
train loss 0.344 test loss 0.415 and accuracy 0.806


In [81]:
train_epocs(model, epochs=10, lr=0.001)

train loss 0.332 test loss 0.408 and accuracy 0.823
train loss 0.329 test loss 0.400 and accuracy 0.831
train loss 0.317 test loss 0.394 and accuracy 0.828
train loss 0.304 test loss 0.395 and accuracy 0.814
train loss 0.300 test loss 0.393 and accuracy 0.812
train loss 0.293 test loss 0.386 and accuracy 0.822
train loss 0.283 test loss 0.377 and accuracy 0.832
train loss 0.273 test loss 0.373 and accuracy 0.835
train loss 0.266 test loss 0.369 and accuracy 0.840
train loss 0.261 test loss 0.366 and accuracy 0.840


In [ ]:
train_epocs(model, epochs=10, lr=0.001)

train loss 0.251 test loss 0.384 and accuracy 0.816
train loss 0.259 test loss 0.364 and accuracy 0.831
train loss 0.243 test loss 0.358 and accuracy 0.841
train loss 0.235 test loss 0.359 and accuracy 0.841
train loss 0.232 test loss 0.354 and accuracy 0.842
train loss 0.223 test loss 0.350 and accuracy 0.844


## Lab 

* Improve tokenization
* Use fasttext instead of globe model. (https://fasttext.cc/docs/en/english-vectors.html)

   `! pip install git+https://github.com/facebookresearch/fastText.git`
* Extend this code to do cross-validation. Look at https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py for an example on how to do it.

## References

The CNN is adapted from here https://github.com/junwang4/CNN-sentence-classification-pytorch-2017/blob/master/cnn_pytorch.py.
Code for the original paper can be found here https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py.